# Introduction and Setup

This Jupyter notebook demonstrates the development of a virtual doctor chatbot that interacts with heart failure patients 
to monitor their progress. The chatbot uses LangChain and GPT-3.5 for natural language processing and response generation.

Components:
1. User Interface (UI): The primary interface for patient interaction.
2. Conversational AI Core (GPT-3.5): Processes user input and generates responses.
3. Dialogue Manager: Manages user sessions and ensures a logical conversation flow.
4. Simulation Toggle: Allows switching between user interaction and simulated patient interaction.

In [112]:
# Import necessary libraries
from typing import List, Callable
import os
from uuid import uuid4
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain.memory import ChatMessageHistory
from typing import List, Callable
import os
from uuid import uuid4
from dotenv import load_dotenv

# Load API keys from .env file
ROOT_DIR = "."  # Set this to the root directory where your .env file is located
load_dotenv(os.path.join(ROOT_DIR, ".env"))

# Set up unique ID for session
unique_id = uuid4().hex[0:8]

# Set environment variables for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "HeartFailureMonitor"

# Define the User Interface

In [125]:
class UserInterface:
    def __init__(self, simulation_mode: bool = False):
        """
        Initialize the User Interface with the given simulation mode.

        Args:
            simulation_mode (bool, optional): Whether to start in simulation mode. Defaults to False.
        """
        self.simulation_mode = simulation_mode  # By default, start in user input mode
    
    def set_simulation_mode(self, simulation_mode: bool):
        """
        Set the simulation mode of the User Interface.

        Args:
            simulation_mode (bool): Whether to start in simulation mode.
        """
        self.simulation_mode = simulation_mode
        mode = "Simulation Mode" if self.simulation_mode else "User Input Mode"
        print(f"Mode toggled to: {mode}")
        
    def collect_user_input(self):
        """
        Collects user input.

        Returns:
            str: The user input.
        """
        if not self.simulation_mode:
            user_input = input("Enter user message. Enter 'exit' to stop chat: ")
            return user_input
        else:
            print("Simulation mode is ON. User input is disabled.")
            return None

    def display_response(self, response: str):
        """
        Displays the response to the user.

        Args:
            response (str): The response to display.
        """
        print(f"Agent's Response: {response}")

# Initialize the UI
ui = UserInterface()

# Define the Dialogue Agent

In [129]:
class DialogueAgent:
    def __init__(self, name: str, system_message: str, model: ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')) -> None:
        """
        Initialize the DialogueAgent with a name, system message, and a language model.
        
        Args:
            name (str): The name of the agent.
            system_message (str): The initial system message to set the context.
            model (ChatOpenAI): The language model to use for generating responses.
        """
        self.name = name
        self.system_message = system_message
        self.model = model

        # Initialize chat message history to keep track of the entire conversation
        self.memory = ChatMessageHistory()
        
        # Define the prompt template with placeholders.
        self.prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content=self.system_message),  # The persistent system prompt
                MessagesPlaceholder(variable_name="chat_history"),  # Where the memory will be stored.
                HumanMessagePromptTemplate.from_template("{human_input}"),  # Where the human input will be injected
            ]
        )
        
        # Define the LLM chain with the model and prompt
        self.chain = LLMChain(
            llm=self.model,
            prompt=self.prompt,
            verbose=True,
        )

        # Initialize the memory with the system message
        self.reset()

    def reset(self):
        """
        Resets the conversation history in memory
        """
        self.memory.clear()

    def send(self) -> str:
        """
        Generates a response based on the conversation history stored in memory.
        
        Returns:
            str: The response generated by the language model.
        """
        # Retrieve the current conversation history as a list of messages
        conversation_history = self.memory.messages

        # Predict the next response using the chain
        response = self.chain.predict(
            chat_history=conversation_history,
            human_input=""  # Empty because we are generating the AI's response
        )

        # Save the AI's response to the memory
        self.memory.add_ai_message(response)
        return response

    def receive(self, name: str, message: str) -> None:
        """
        Adds a new message to the conversation history in memory.
        
        Args:
            name (str): The name of the sender (usually "Patient" or "Doctor").
            message (str): The content of the message.
        """
        # Save the user input to the conversation memory
        self.memory.add_user_message(message)
    
    def get_history(self) -> List[str]:
        """
        Retrieves the full conversation history stored in memory.
        
        Returns:
            List[str]: The list of messages in the conversation history.
        """
        formatted_history = []
        for msg in self.memory.messages:
            if isinstance(msg, HumanMessage):
                formatted_history.append(f"User: {msg.content}")
            elif isinstance(msg, AIMessage):
                formatted_history.append(f"AI: {msg.content}")
            else:
                formatted_history.append(f"System: {msg.content}")
        return formatted_history

## Test dialogue agent

In [131]:
# Test function for the DialogueAgent with full conversation export
def test_dialogue_agent(
        system_message = "You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.",
        messages = ["My name is Gary Kong", "I am not feeling well", "Remind me what my name is"]
        ):
    """
    Tests the DialogueAgent acting as a doctor with a series of patient messages.
    This can also be modified to make the agent act as a patient and the messages be doctor messages.

    Args:
        system_message (str): The initial system message for the agent
        messages (List[str]): The list of messages to test the agent with
    """
    # Set up the OpenAI model
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        name="Doctor",
        system_message=system_message,
        model=model,
    )
    
    # Reset the conversation
    agent.reset()
        
    # Simulate the conversation
    for msg in messages:
        # Doctor receives the patient's message
        agent.receive(name="Patient", message=msg)
        
        # Doctor sends a response
        response = agent.send()
        
        # Print the response
        print(f"User: {msg}")
        print(f"AI: {response}\n")

    # Display the full conversation history
    print("\nFull Conversation History:")
    for message in agent.get_history():
        print(message)

# Run the test function
test_dialogue_agent()



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.
Human: My name is Gary Kong
Human: 

> Finished chain.
User: My name is Gary Kong
AI: Hello Gary! How can I assist you today regarding your heart failure?



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.
Human: My name is Gary Kong
AI: Hello Gary! How can I assist you today regarding your heart failure?
Human: I am not feeling well
Human: 

> Finished chain.
User: I am not feeling well
AI: I'm sorry to hear that, Gary. It's important to monitor your symptoms closely and contact your healthcare provider for further evaluation and management. Take care of yourself and don't hesitate to seek medical attention if needed.



> Entering new LLMChain ch

In [132]:
# Integration and testing function
def test_dialogue_with_user_interface(system_message = "You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates."):
    """
    This function simulates a conversation between a virtual doctor and a patient using a user interface, whereby the user can input messages acting as the patient.
    This can also be modified to simulate a conversation whereby the agent acts as the patient and the user acts as the doctor.

    Args:
        system_message (str, optional): The system message for the agent. Defaults to "You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.".
    """
    # Set up the OpenAI model
    model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')
    
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        name="Dr. Alexis Wang",
        system_message=system_message,
        model=model
    )
    
    # Create the UserInterface instance
    ui = UserInterface(simulation_mode=False)  # Start in user input mode
    
    # Simulate the interaction
    while True:
        # Collect user input
        user_input = ui.collect_user_input()
        
        if user_input.lower() == 'exit':
            break  # Exit if in simulation mode or if input is None
        
        # Doctor agent receives the user's message
        agent.receive(name="Patient", message=user_input)
        
        # Doctor agent sends a response
        response = agent.send()
        
        # Display the doctor's response
        ui.display_response(response)
    
    # Display the full conversation history
    print("\nFull Conversation History:")
    for message in agent.get_history():
        print(message)

# Run the test function
test_dialogue_with_user_interface()



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.
Human: My name is Gary
Human: 

> Finished chain.
Agent's Response: Hello Gary! How are you feeling today?


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.
Human: My name is Gary
AI: Hello Gary! How are you feeling today?
Human: Unwell
Human: 

> Finished chain.
Agent's Response: I'm sorry to hear that, Gary. Have you been experiencing any specific symptoms or changes in your condition recently?


> Entering new LLMChain chain...
Prompt after formatting:
System: You are Dr. Alexis Wang, a virtual doctor specialized in heart failure. Respond concisely and empathetically to patient updates.
Human: My name is Gary
AI: Hello Gary! How are you feeling 

# Define dialogue simulator